In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.colab.data_table

In [ ]:
project_id = 'trusty-shine-430910-t0'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/Nested Queries**

In [ ]:
df = client.query('''
  SELECT
  `hash` AS transaction_hash,
  block_hash
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe")
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash
0,0x24dbcd8bcede8b76eee70d78d268dcc35010d980d859...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
1,0xdca9ab31d180f7c7e1a8b3164b594bacd8f7b1d8a884...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
2,0x94f0048e982a4afbed20dce69f61b42adc46764f04a1...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
3,0x98d64143cad58597c62ee845d96b87b0fea2c3a3e774...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
4,0xf2654bf824f2064ce0c638902d008cc3096633b06b30...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...


**Joins**

In [ ]:
df = client.query('''
  SELECT
  t.transaction_hash,
  t.block_hash,
  tb.parent_hash
FROM (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")) AS t
INNER JOIN (
  SELECT
    `hash` AS block_hash,
    parent_hash
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe" ) AS tb
ON
  t.block_hash = tb.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash,parent_hash
0,0x00f2d3227b470e4d31d2066bd4215fed669a21e5a570...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
1,0x01606a9bb282c68fa477662c2b6ccca758b9a1782448...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
2,0x018df5d15cf63ba4fc980b20ba54a023b4c2b24c67aa...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
3,0x0420d46f8d2027d43b3fce1fb4d34884bfdf226596e0...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
4,0x047800c856069dc51b0e90230884a79660e415b1cb33...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...


**Unions**

In [ ]:
df = client.query('''
  (SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_usa.population_by_zip_2000`
LIMIT
  10)
UNION DISTINCT
(SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_usa.population_by_zip_2010`
LIMIT
  10)''').to_dataframe()

df.head()

,geo_id
0,8600000US25647
1,8600000US49336
2,8600000US72106
3,8600000US74074
4,8600000US01960


**Aggregate Functions**

In [ ]:
df = client.query('''
  SELECT
  block_hash,
  COUNT(`hash`) AS total_transaction
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
GROUP BY
  (block_hash)''').to_dataframe()

df.head()

,block_hash,total_transaction
0,0xe023a41df70f6380b8874ef5aec600ef488b708d8aa1...,282
1,0xbccb030bc9162b3c57afae6dff5492a0ae5c3c93a917...,63
2,0x128e63bb1c98c95e49ca628a454757252f01745ad695...,230
3,0x16fd369f16c03ea3da2d5ed3f47a60a9887862f6f762...,198
4,0xd56f41240dfdbbbc2f6ae74726e9b395518a1f5741cc...,109


**Window Functions**

In [ ]:
df = client.query('''
SELECT
  block_hash,
  total_transaction,
  ROW_NUMBER() OVER (ORDER BY total_transaction DESC) AS rank
FROM (
  SELECT
    block_hash,
    COUNT(`hash`) AS total_transaction
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND block_hash IN (
    SELECT
      `hash`
    FROM
      `bigquery-public-data.crypto_ethereum.blocks`
    WHERE
      TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
  GROUP BY
    (block_hash))''').to_dataframe()

df.head()

,block_hash,total_transaction,rank
0,0xd21124debd7f04c43d8c2697b0f4995664ebd69ebabd...,894,1
1,0xad1bb588af8d932a91e2b4b875dccaff641c0590c286...,777,2
2,0xfab03139f33dff846394e076e6e0d4f4c1c4cf2f7984...,767,3
3,0xb18712c4f93c81186f1cda73ee52e577aabaa2b51a63...,763,4
4,0x02e720d3d306f3eacb0ebbae3ea589e1b4e32e64937b...,733,5


**Common Table Expressions (CTEs)**

In [ ]:
df = client.query('''
WITH
  transaction_block AS (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")),
  block_parent AS (
  SELECT
    `hash` AS block_hash,
    parent_hash
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe" )
SELECT
  transaction_block.transaction_hash,
  transaction_block.block_hash,
  block_parent.parent_hash
FROM
  transaction_block
INNER JOIN
  block_parent
ON
  transaction_block.block_hash = block_parent.block_hash
ORDER BY
  transaction_block.transaction_hash
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash,parent_hash
0,0x00f2d3227b470e4d31d2066bd4215fed669a21e5a570...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
1,0x01606a9bb282c68fa477662c2b6ccca758b9a1782448...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
2,0x018df5d15cf63ba4fc980b20ba54a023b4c2b24c67aa...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
3,0x0420d46f8d2027d43b3fce1fb4d34884bfdf226596e0...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
4,0x047800c856069dc51b0e90230884a79660e415b1cb33...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...


**Pivot**

In [ ]:
%%bigquery --project trusty-shine-430910-t0
WITH Produce AS (
  SELECT 'Banana' as product, 51 as sales, 'Q1' as quarter, 2020 as year UNION ALL
  SELECT 'Banana', 23, 'Q2', 2020 UNION ALL
  SELECT 'Banana', 45, 'Q3', 2020 UNION ALL
  SELECT 'Banana', 3, 'Q4', 2020 UNION ALL
  SELECT 'Banana', 70, 'Q1', 2021 UNION ALL
  SELECT 'Banana', 85, 'Q2', 2021 UNION ALL
  SELECT 'Apple', 77, 'Q1', 2020 UNION ALL
  SELECT 'Apple', 0, 'Q2', 2020 UNION ALL
  SELECT 'Apple', 1, 'Q1', 2021)
SELECT * FROM Produce

Query is running:   0%|          |

Downloading:   0%|          |

,product,sales,quarter,year
0,Banana,51,Q1,2020
1,Banana,23,Q2,2020
2,Banana,45,Q3,2020
3,Banana,3,Q4,2020
4,Banana,70,Q1,2021
5,Banana,85,Q2,2021
6,Apple,77,Q1,2020
7,Apple,0,Q2,2020
8,Apple,1,Q1,2021


In [ ]:
%%bigquery --project trusty-shine-430910-t0
WITH Produce AS (
  SELECT 'Banana' as product, 51 as sales, 'Q1' as quarter, 2020 as year UNION ALL
  SELECT 'Banana', 23, 'Q2', 2020 UNION ALL
  SELECT 'Banana', 45, 'Q3', 2020 UNION ALL
  SELECT 'Banana', 3, 'Q4', 2020 UNION ALL
  SELECT 'Banana', 70, 'Q1', 2021 UNION ALL
  SELECT 'Banana', 85, 'Q2', 2021 UNION ALL
  SELECT 'Apple', 77, 'Q1', 2020 UNION ALL
  SELECT 'Apple', 0, 'Q2', 2020 UNION ALL
  SELECT 'Apple', 1, 'Q1', 2021)
SELECT * FROM Produce
PIVOT(SUM(sales) FOR quarter IN ('Q1', 'Q2', 'Q3', 'Q4'))

Query is running:   0%|          |

Downloading:   0%|          |

,product,year,Q1,Q2,Q3,Q4
0,Banana,2020,51,23,45,3
1,Banana,2021,70,85,<NA>,<NA>
2,Apple,2020,77,0,<NA>,<NA>
3,Apple,2021,1,<NA>,<NA>,<NA>


**String Manipulation**

Concat

In [13]:
%%bigquery --project trusty-shine-430910-t0
SELECT
  DISTINCT CONCAT(from_address, to_address) AS address
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
LIMIT
  10

Query is running:   0%|          |

Downloading:   0%|          |

,address
0,0xf8b483dba2c3b7176a3da549ad41a48bb31210690x46...
1,0x74270bb4aadfd6786d7c404ca3ac4188d4c52bb60xb2...
2,0x507e1dcfa920f2f5fa495d79dfe24ff7d98b45560x1e...
3,0x6f79bc4329bc1d5a387077fc999aff8ca0e3d0f70x49...
4,0xbf94f0ac752c739f623c463b5210a7fb2cbb420b0xd1...
5,0x4976a4a02f38326660d17bf34b431dc6e2eb23270x8e...
6,0x1689a089aa12d6cbbd88bc2755e4c192f87020000xf4...
7,0x3cd6bf92ee255d6b0637aa0d4dc7bb265ced07c30xe3...
8,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f5110x1e...
9,0xf22235e9542de0ad61052776517ceda5f78b7c660xad...


**Date and Time**

In [15]:
%%bigquery --project trusty-shine-430910-t0
SELECT
  TIMESTAMP_TRUNC(block_timestamp, DAY) AS tx_date
FROM
  bigquery-public-data.crypto_ethereum.transactions
WHERE
  block_timestamp >= CAST(DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) AS TIMESTAMP)
LIMIT
  10

Query is running:   0%|          |

Downloading:   0%|          |

,tx_date
0,2024-10-06 00:00:00+00:00
1,2024-10-06 00:00:00+00:00
2,2024-10-06 00:00:00+00:00
3,2024-10-06 00:00:00+00:00
4,2024-10-06 00:00:00+00:00
5,2024-10-06 00:00:00+00:00
6,2024-10-06 00:00:00+00:00
7,2024-10-06 00:00:00+00:00
8,2024-10-06 00:00:00+00:00
9,2024-10-06 00:00:00+00:00


**Case Statement**

In [16]:
%%bigquery --project trusty-shine-430910-t0
SELECT
  `hash` AS transaction_hash,
  gas_price,
  CASE
    WHEN gas_price >= 100000000000 THEN 'High Gas Price'
    ELSE 'Low Gas Price'
END
  AS gas_price_category
FROM
  bigquery-public-data.crypto_ethereum.transactions
LIMIT
  10

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_hash,gas_price,gas_price_category
0,0x074790becdc1b07dfc3087bca63dbf5a6cfea4f7ab8e...,10000000000,Low Gas Price
1,0xf616cbc43baddffd5315d479418c6440c0edd165f32c...,4972520000,Low Gas Price
2,0xb0529e3d24689b4a737fefdafada24f5da8459fe6916...,6235095876,Low Gas Price
3,0x1cf1c9427e3412c6f4c20541b1c3c819eb80a16cb62c...,4313393051,Low Gas Price
4,0x09075f41b0aac8496f2727c46e4aea7eb2a31751141a...,4972520000,Low Gas Price
5,0x4414d04993d44cedc8b6a85dbb46abd2bbd225322784...,4377268432,Low Gas Price
6,0xadc5337ba5bdeeb56801f1fa9d57aa22846d3db13ad3...,4531552587,Low Gas Price
7,0xa362b64fe0165e84cd8ebce3d8af315cfc43f4a73418...,6377268432,Low Gas Price
8,0xebed483e77c8ea96575f622851fad53a072fda75d9b8...,5114692556,Low Gas Price
9,0x0215500fc8c3a87214ea396f5c1eb75de233ed0b5533...,4995326932,Low Gas Price


In [18]:
%%bigquery --project trusty-shine-430910-t0
CREATE TEMP FUNCTION
  getCategory(arr ANY TYPE) AS (
    CASE
      WHEN arr >= 100000000000 THEN 'High Gas Price'
      ELSE 'Low Gas Price'
  END
    );
SELECT
  gas_price,
  getCategory(gas_price) AS category
FROM (
  SELECT
    gas_price
  FROM
    bigquery-public-data.crypto_ethereum.transactions
  LIMIT
    10)

Query is running:   0%|          |

Downloading:   0%|          |

,gas_price,category
0,3777738363,Low Gas Price
1,4517023296,Low Gas Price
2,4539918707,Low Gas Price
3,4517023296,Low Gas Price
4,15000000000,Low Gas Price
5,24740782824,Low Gas Price
6,4030941478,Low Gas Price
7,4030941478,Low Gas Price
8,4083941478,Low Gas Price
9,5507855426,Low Gas Price


**Recursive Queries**

In [19]:
%%bigquery --project trusty-shine-430910-t0
WITH
  RECURSIVE CTE_1 AS ( (
    SELECT
      1 AS iteration
    UNION ALL
    SELECT
      1 AS iteration)
  UNION ALL
  SELECT
    iteration + 1 AS iteration
  FROM
    CTE_1
  WHERE
    iteration < 3 )
SELECT
  iteration
FROM
  CTE_1
ORDER BY
  1 ASC

Query is running:   0%|          |

Downloading:   0%|          |

,iteration
0,1
1,1
2,2
3,2
4,3
5,3


**Temporary Tables**

In [21]:
%%bigquery --project trusty-shine-430910-t0
CREATE TEMPORARY TABLE temp_transactions ( transaction_id STRING,
    gas_price INT );
INSERT INTO
  temp_transactions
SELECT
  `hash`,
  gas_price
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
LIMIT
  10;

Executing query with job ID: 2b902a61-ab84-4868-98fa-f2d709a94c5e
Query executing: 0.15s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/trusty-shine-430910-t0/queries/2b902a61-ab84-4868-98fa-f2d709a94c5e?maxResults=0&location=US&prettyPrint=false: Billing has not been enabled for this project. Enable billing at https://console.cloud.google.com/billing. DML queries are not allowed in the free tier. Set up a billing account to remove this restriction. at [3:1]

Location: US
Job ID: 2b902a61-ab84-4868-98fa-f2d709a94c5e

